Now we perform a baseline training. Our model will be an Temporal Fusion Transformer (TFT). We use the implementation provided by PyTorch Forecast. Our training set will be from January 1970 to December 2009. Our test set will be from January 2010 to December 2024.

There are multiple methods to estimate if a stock is a value trap. The most principled is to perform an ordinal regression on the TTM Price Change. This is also the most difficult. The TFT model outputs quantiles, and so we leverage these quantiles to estimate the ordinal positions of the stocks. We outline the method as follows:
* Select quantiles $q_1, \ldots, q_n$.
* For each company $i$, receive predictions $\hat{y_i} = (\hat{y_i}(q_1), \ldots, \hat{y_i}(q_n))$.
* Choose an ordering of $[n]$.
* Order $\hat{y_i}$ lexicographically.

The quantiles selected and the ordering of $[n]$ are chosen via cross-validation. For simplicity, we will only use two quantiles, however in practice, cross validation for both of these is strongly reccommended, but computationally expensive.



We provide descriptions for each of the features included in our dataset. If there is a q at the end of a feature name, this indicates this data is reported quarterly. If there is a y at the end of a feature name, this indicates, the data is reported yearly.

* time: An integer that represents the current quarter. 0 for the first quarter in the dataset. Increments by 1 for each quarter.
* gvkey: Unique company id
* datadate: Year and month
* fqtr: Financial Quarter
* indfmt: Style of financial reports. All will be industrial              
* consol: Level of consolidation. All will be consolidation           
* popsrc: Population Source. All will be domestic/US
* datafmt: Data Format.
* curcdq: Currency. All will be USD
* actq: Current Assets Total
* apq: Accounts Payable
* atq: Assets total
* cheq: Cash and short term investments
* cogsq: Cost of Goods Sold
* cshoq: Common Shares Outstanding
* dlcq: Debt in Current Liabilities
* dlttq: Long Term Debt - Total
* intanq: Intangible Assets
* lctq: Current Liabilities - Total
* ltq: Liabilities - Total
* mibq: Noncontrolling Interest
* niq: Net Income (Loss)
* oiadpq: Operating Income After Depreciation
* ppentq: Property, Plant, and Equipment - Total
* pstkq: Preferred/Preference Stock - Total
* rectq: Receivables Total
* saleq: Sales/Turnover (Net)
* seqq: Shareholders' Equity.
* capxy: Capital Expenditures
* cdvcy: Cash Dividends on Common Stock
* exchg: Exchange on Which Stock is Traded On. Either NYSE, NASDAQ, or AMEX
* costat: Postal Code Character
* prccq: Price Close
* prclq: Price Low
* loc: Location of Company Headquarters. Country. Will always be USA
* ipodate: Date of IPO. Is NaN if no IPO or date is missing for other reason.
* ret_3m: Percent Price Change over last 3 months
* ret_9m: Percent Price Change over last 9 months
* ret_12m: Percent Price Change over last 12 months
* ret_18m: Percent Price Change Over Last 18 Months
* bkmktq: Book to Market.
$$\text{Book to Market } = \frac{\text{Shareholder Equity}}{\text{Closing Price } \cdot \text{ Common Shares Outstanding}} $$
* rel:bkmktq: Percentile by Book to Market
* evq: Enterprise Value
\begin{align*}
\text{Enterprise Value} &= \text{Closing Price } \cdot \text{ Common Shares Outstanding}\\
 &+ \text{ Long Term Debt } + \text{ Debt in Current Liabilities}\\
 &- \text{ Cash and Short Term Investments}
\end{align*}
* eyq: Earnings Yield:
$$
\text{Earnings Yield} = \frac{\text{Operating Income After Depreciation}}{\text{Enterprise Value}}
$$
* rel:eyq: Percentile by Earnings Yield

The next set of variables are just indicators for if a given entry is missing. Refer to the NaN policy to see how missing values are treated.

The next set of variables are the seasonalized variables. Seasonalization is only applied to the variables that are continuous. We list them here for posterity. Refer to the Seasonalizaiton section to see how this is implemented.
* szn:actq            
* szn:apq             
* szn:atq             
* szn:cheq            
* szn:cogsq           
* szn:cshoq           
* szn:dlcq            
* szn:dlttq           
* szn:intanq          
* szn:lctq            
* szn:ltq             
* szn:mibq            
* szn:niq             
* szn:oiadpq          
* szn:ppentq          
* szn:pstkq           
* szn:rectq           
* szn:saleq           
* szn:seqq            
* szn:capxy           
* szn:cdvcy           
* szn:prccq           
* szn:prclq           
* szn:ret_3m          
* szn:rel:ret_3m      
* szn:ret_9m          
* szn:rel:ret_9m      
* szn:ret_12m         
* szn:rel:ret_12m     
* szn:ret_18m         
* szn:rel:ret_18m     
* szn:bkmktq          
* szn:rel:bkmktq      
* szn:evq             
* szn:eyq             
* szn:rel:eyq         

---

We can divide this features into five categories:
* Static Categoricals: Features like gvkey
* Time Varying Known Categoricals: Features like the financial quarter.
* Time Varying Known Reals: IPODate is the only one.
* Time Varying Unknown Categoricals: NaN features
* Time Varying Unknown Reals: All of the rest of the features.

In [1]:
import numpy as np
import pandas as pd
import typing
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -q pytorch-forecasting
from pytorch_forecasting import TimeSeriesDataSet
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
train = torch.load('/content/drive/MyDrive/Colab Notebooks/ECMA 31330 Project/train.pt', weights_only = False)
test = torch.load('/content/drive/MyDrive/Colab Notebooks/ECMA 31330 Project/test.pt', weights_only = False)
batch_size = 1024
train_dataloader = train.to_dataloader(train=True, batch_size=batch_size, num_workers=8)
val_dataloader = train.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=8)
test_dataloader = test.to_dataloader(train=False)

In our model we feed in 20 quarters of time series data for a company. We output 20 quarters of results. These results are the probability that in that quarter, the stock is a value trap.

The following is a list of all the hyperparameters we need for our TFT model. We note hyperparameters that are chosen with cross validation with an asterisk.

```python
* hidden_size # Number of nodes in a layer
lstm_layers = 2
* dropout_rate
output_size = 2 # necessary for CrossEntropy metric
loss = CrossEntropy()
attention_head_size = 4 # Number of Attention Heads
* hidden_continous_sizes # <= hidden_size. Num nodes set aside for cts features
* learning_rate
* reduce_on_plateau_patience
* gradient_clip_val
```



In [11]:
from pytorch_forecasting.models import (
    BaseModel, TemporalFusionTransformer
)
from pytorch_forecasting.metrics import (
    QuantileLoss, CrossEntropy
)
import lightning.pytorch as pl
from lightning.pytorch.callbacks import (
    EarlyStopping, LearningRateMonitor
)
from lightning.pytorch.loggers import TensorBoardLogger

!pip uninstall tensorflow
!pip install -U tensorboard
!pip install pytorch_optimizer

import pytorch_optimizer

In [12]:
import os
num_layers = 2
hidden_size = 16
attention_head_size = 4
hidden_continuous_size = 8
droput = 0.1
output_size = 2
loss = CrossEntropy()
learning_rate = 0.001
reduce_on_plateau_patience = True
epochs = 10
# configure network and trainer
early_stop_callback = EarlyStopping(
    monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min"
)
lr_logger = LearningRateMonitor()  # log the learning rate
log_dir = os.path.join(os.getcwd(), "lightning_logs")  # Create a directory if it doesn't exist
os.makedirs(log_dir, exist_ok=True)
logger = TensorBoardLogger(log_dir)  # logging results to a tensorboard

In [13]:
tft = TemporalFusionTransformer.from_dataset(
  train,
  hidden_size = hidden_size,
  lstm_layers=num_layers,
  dropout=droput,
  output_size=output_size,
  loss=loss,
  attention_head_size=attention_head_size,
  hidden_continuous_size=hidden_continuous_size,
  learning_rate=learning_rate,
  optimizer = 'ranger',
  log_interval=0
)

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [14]:
tft_trainer = pl.Trainer(
    accelerator='auto',
    devices='auto',
    check_val_every_n_epoch=10,
    # fast_dev_run=True,
    max_epochs=epochs,
    max_time={'minutes':30},
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
    enable_model_summary=True,
    enable_checkpointing=True,
    gradient_clip_val=0.5
)

INFO: You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:lightning.pytorch.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [15]:
torch.set_float32_matmul_precision('medium')
tft_trainer.fit(tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | CrossEntropy                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 185 K  | train
3  | prescalers                         | ModuleDict                      | 1.1 K  | train
4  | static_variable_selection          | VariableSelectionNetwork        | 574    | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 54.1 K | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 1.4 K  | train
7  | static_context_variable_selectio

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/loops/training_epoch_loop.py:464: ReduceLROnPlateau conditioned on metric val_loss which is not available but strict is set to `False`. Skipping learning rate update.
INFO: Time limit reached. Elapsed time is 0:30:02. Signaling Trainer to stop.
INFO:lightning.pytorch.utilities.rank_zero:Time limit reached. Elapsed time is 0:30:02. Signaling Trainer to stop.


In [19]:
tft_trainer.save_checkpoint('/content/drive/MyDrive/Colab Notebooks/ECMA 31330 Project/tft.pt')

In [ ]:
tft